In [ ]:
from main_cnn import main
import argparse
import numpy as np

In [ ]:
parser = argparse.ArgumentParser()
parser.add_argument('--config', help='Path to the config file', default='./default_config.yml')
parser.add_argument('--log-dir', help='Path to the log directory', default='../log/')
parser.add_argument('--posters', help='Path to the posters', default='../data/posters/')
parser.add_argument('--models-dir', help='Path to the saved models', default='../data/models/')
parser.add_argument('--sets-dir', help='Path to the training and testing sets', default='../data/sets/')
parser.add_argument('--database', help='Path to the databse csv', default='../data/poster_data.csv')
parser.add_argument('--csv', help='Path to the clean csv', default='../data/')
parser.add_argument('-s', '--save', help='Save model', action='store_true')
parser.add_argument('-v', '--verbose', help='Verbose', action='store_true')

str_args = '-v -s'.split()
args, _ = parser.parse_known_args(str_args)

In [ ]:
movies, model, train_posters, train_genres, train_ids, test_posters, test_genres, test_ids, clean_movies, predicted_genres, training_history = main(args)

In [ ]:
from utils.display.training_display import plot_loss
%matplotlib inline

if training_history is not None:
    plot_loss(training_history)

In [ ]:
import yaml

config = yaml.safe_load(open(args.config))
genres_inv = {config['genres'][k]: k for k in config['genres'].keys()}
print(genres_inv)

In [ ]:
# FOR MULTI LABEL
# import numpy as np

# predictions = [] 
# for pred in predicted_genres:
#     first3_index = np.argsort(pred)[-3:]
#     predictions.append([genres_inv[k] for k in first3_index])
# ground_truth = []
# for truth in test_genres:
#     first3_index = np.nonzero(truth)[0]
#     ground_truth.append([genres_inv[k] for k in first3_index])

predictions = np.array([genres_inv[k] for k in np.argmax(predicted_genres, axis=1)])
ground_truth = np.array([genres_inv[k] for k in np.argmax(test_genres, axis=1)])

In [ ]:
movies.set_index('allocine_id', inplace=True)

In [ ]:
r = np.random.permutation(100)[:10]
for i in r:
    print('{}: truth: {}, pred: {}'.format(movies.loc[test_ids[i], 'title'], ground_truth[i], predictions[i]))
#     print('Ground truth: {}, real: {}'.format(ground_truth[i], movies.loc[test_ids[i], 'genres']))

In [ ]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

In [ ]:
from utils.display.results_display import plot_test_results

starting_index = 0
num_row, num_cols = 10, 4
plot_test_results(test_posters, test_genres, genres_inv, predicted_genres, starting_index, num_row, num_cols)